In [1]:
#Python Basics
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import string
import pickle
import requests
from bs4 import BeautifulSoup
import lyricsgenius

#NLP & ML
import editdistance as ed
from sklearn.pipeline import Pipeline
import gensim
import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer


clear_output(wait=True)

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
clear_output(wait=True)

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:1108)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


In [3]:
#API Credentials
client_id = 'INSERT YOUR OWN'
client_secret = 'INSERT YOUR OWN'
client_access_token = 'INSERT YOUR OWN'

In [110]:
kw_disco = pd.read_csv("Kanye West Discography.csv")
kw_disco['track'] = kw_disco['track'].apply(lambda x: x.replace(u'\xa0', u' '))
kanye = kw_disco
eng_stopwords = set(stopwords.words("english"))

kanye[['clean_track', 'features']] = pd.DataFrame(kanye['track'].str.split(' \(featuring ').values.tolist())
kanye['features'] = kanye['features'].str.replace('\)', '')
kanye['lyrics'] = 'NA'
kanye.head()

,albums,track_number,track,writers,track_length,Year,old_kanye,clean_track,features,lyrics
0,The College Dropout,1,Intro (Skit),Kanye West,0:19,2004,1,Intro (Skit),None,NA
1,The College Dropout,2,We Don't Care,West Miri Ben-Ari Ross Vannelli,3:59,2004,1,We Don't Care,None,NA
2,The College Dropout,3,Graduation Day,West John Stephens Ben-Ari,1:22,2004,1,Graduation Day,None,NA
3,The College Dropout,4,All Falls Down (featuring Syleena Johnson),West Lauryn Hill,3:43,2004,1,All Falls Down,Syleena Johnson,NA
4,The College Dropout,5,I'll Fly Away,Albert E. Brumley,1:09,2004,1,I'll Fly Away,None,NA


In [62]:
genius = lyricsgenius.Genius(client_access_token)
artist = genius.search_artist("Kanye West", max_songs=1)

def lyrics_search(track, artist):
    try:
        return genius.search_song(track, artist).lyrics
    except AttributeError:
        return 'No Lyrics'

def second_search(track, artist):
    original_track = track
    track = track.replace(' ', '%20').strip()
    artist = artist.replace(' ', '%20').strip()
    client_access_token = '7sJP8jxBp5rvBePVOmWUHlfP7oDQQQxhBUyQuRtGR0wrtxRdq7ucFtE04vnp38Uk'
    
    search_url = 'https://api.genius.com/search?q=' + track + '%20' + artist + '&access_token=' + client_access_token
    response = requests.get(search_url)
    json = response.json() 
    
    for i in range(10):
        json_title = json['response']['hits'][i]['result']['title']
        if ed.eval(json_title, original_track) < 5:
            
            page_url = json['response']['hits'][i]['result']['url']
            page = requests.get(page_url)

            html = BeautifulSoup(page.text, 'html.parser')
            lyrics = html.find("div", class_="lyrics").get_text()
            return lyrics
        else:
            return 'No Lyrics'

def third_search(track):
    lookup = {'Intro (Skit)' : 'https://genius.com/Kanye-west-intro-lyrics', 
                    'Workout Plan (Skit)' : 'https://genius.com/Kanye-west-workout-plan-lyrics',
                    'Skit No. 1' : 'https://genius.com/Kanye-west-skit-1-lyrics',
                    'Skit No. 2' : 'https://genius.com/Kanye-west-skit-2-lyrics',
                    'Skit No. 3' : 'https://genius.com/Kanye-west-skit-3-lyrics',
                    'Mercy' : 'https://genius.com/Kanye-west-big-sean-pusha-t-and-2-chainz-mercy1-lyrics',
                    'New God Flow' : 'https://genius.com/Kanye-west-pusha-t-and-ghostface-killah-new-god-flow1-lyrics',
                    'Ye vs. the People':'https://genius.com/Kanye-west-ye-vs-the-people-starring-ti-as-the-people-lyrics',
                    'Water (featuring Ant Clemons)' : 'https://genius.com/Kanye-west-water-lyrics'}
    if track in lookup.keys():
        search_url =  lookup[track]
        response = requests.get(search_url)
        html = BeautifulSoup(response.text, "html.parser")
        lyrics = html.find("div", class_="lyrics").get_text()
        return lyrics
        

Searching for songs by Kanye West...

Song 1: "Mercy"

Reached user-specified song limit (1).
Done. Found 1 songs.


In [51]:
track = 'Im in it'
artist = 'Kanye West'
track = track.replace(' ', '%20').strip()
artist = artist.replace(' ', '%20').strip()
client_access_token = '7sJP8jxBp5rvBePVOmWUHlfP7oDQQQxhBUyQuRtGR0wrtxRdq7ucFtE04vnp38Uk'

search_url = 'https://api.genius.com/search?q=' + track + '%20' + artist + '&access_token=' + client_access_token
response = requests.get(search_url)
json = response.json()

page_url = json['response']['hits'][0]['result']['url']
page = requests.get(page_url)

html = BeautifulSoup(page.text, 'html.parser')
lyrics = html.find("div", class_="lyrics").get_text()

In [5]:
albums = list(kanye['albums'].unique())
albums

['The College Dropout',
 'Late Registration',
 'Graduation',
 '808s & Heartbreak',
 'My Beautiful Dark Twisted Fantasy',
 'Cruel Summer',
 'Yeezus',
 'The Life of Pablo',
 'Ye',
 'Singles',
 'Cruel Winter',
 'Jesus is King']

In [26]:
#If API doesn't freeze: kanye['lyrics'] = kanye['clean_track'].apply(lambda x : lyrics_search(x, 'Kanye West'))

#Chunking it out cuz it breaks: Maybe need to run this in chunks if API keeps freezing i.e. 0-9, 10-19
for album in albums:
    kanye.loc[kanye['albums']==album, 'lyrics'] = kanye.loc[kanye['albums']==album, 'clean_track'].apply(lambda x : lyrics_search(x, 'Kanye West'))

kanye.head()    

Searching for "Intro (Skit)" by Kanye West...
Specified song was not first result
Searching for "We Don't Care" by Kanye West...
Specified song was not first result
Searching for "Graduation Day" by Kanye West...
Done.
Searching for "All Falls Down" by Kanye West...
Done.
Searching for "I'll Fly Away" by Kanye West...
Specified song was not first result
Searching for "Spaceship" by Kanye West...
Done.
Searching for "Jesus Walks" by Kanye West...
Done.
Searching for "Never Let Me Down" by Kanye West...
Done.
Searching for "Get Em High" by Kanye West...
Done.
Searching for "Workout Plan (Skit)" by Kanye West...
Specified song was not first result
Searching for "The New Workout Plan" by Kanye West...
Done.
Searching for "Slow Jamz" by Kanye West...
Specified song was not first result
Searching for "Breathe in Breathe Out" by Kanye West...
Done.
Searching for "School Spirit (Skit 1)" by Kanye West...
Specified song does not contain lyrics. Rejecting.
Searching for "School Spirit" by Kanye 

,albums,track_number,track,writers,track_length,Year,old_kanye,clean_track,features,lyrics
0,The College Dropout,1,Intro (Skit),Kanye West,0:19,2004,1,Intro (Skit),None,No Lyrics
1,The College Dropout,2,We Don't Care,West Miri Ben-Ari Ross Vannelli,3:59,2004,1,We Don't Care,None,No Lyrics
2,The College Dropout,3,Graduation Day,West John Stephens Ben-Ari,1:22,2004,1,Graduation Day,None,"[Spoken: DeRay]\nWhat in the fuck was that, Ka..."
3,The College Dropout,4,All Falls Down (featuring Syleena Johnson),West Lauryn Hill,3:43,2004,1,All Falls Down,Syleena Johnson,"[Chorus: Syleena Johnson & Kanye West]\nOh, wh..."
4,The College Dropout,5,I'll Fly Away,Albert E. Brumley,1:09,2004,1,I'll Fly Away,None,No Lyrics


In [ ]:
#Checkpoint kanye.to_pickle('kanye_first_search.pkl')
#Manually stitching albums if needed
#album_of_choice = ''
#kanye.loc[kanye['albums']==album_of_choice, 'lyrics'] = kanye.loc[kanye['albums']==album_of_choice, 'clean_track'].apply(lambda x : lyrics_search(x, 'Kanye West'))

In [69]:
kanye.loc[kanye['lyrics'] == 'No Lyrics', 'lyrics'] = kanye.loc[kanye['lyrics'] == 'No Lyrics', 'clean_track'].apply(
                                                        lambda x : second_search(x, 'Kanye West'))


In [70]:
kanye.loc[kanye['lyrics'] == 'No Lyrics',]

,albums,track_number,track,writers,track_length,Year,old_kanye,clean_track,features,lyrics
0,The College Dropout,1,Intro (Skit),Kanye West,0:19,2004,1,Intro (Skit),None,No Lyrics
9,The College Dropout,10,Workout Plan (Skit),West,0:46,2004,1,Workout Plan (Skit),None,No Lyrics
25,Late Registration,5,Skit No. 1,West,0:33,2005,1,Skit No. 1,None,No Lyrics
32,Late Registration,12,Skit No. 2,West,0:31,2005,1,Skit No. 2,None,No Lyrics
35,Late Registration,15,Skit No. 3,West,0:24,2005,1,Skit No. 3,None,No Lyrics
82,Cruel Summer,3,New God Flow (featuring Pusha T),Kanye West Terrence Thornton Herb Rooney Ronal...,5:57,2012,1,New God Flow,Pusha T,No Lyrics
122,Singles,5,Ye vs. the People (featuring T.I.),West Harris Dozier Holland Holland Jr.,3:22,2018,0,Ye vs. the People,T.I.,No Lyrics


In [74]:
lookup = {'Intro (Skit)' : 'https://genius.com/Kanye-west-intro-lyrics', 
                    'Workout Plan (Skit)' : 'https://genius.com/Kanye-west-workout-plan-lyrics',
                    'Skit No. 1' : 'https://genius.com/Kanye-west-skit-1-lyrics',
                    'Skit No. 2' : 'https://genius.com/Kanye-west-skit-2-lyrics',
                    'Skit No. 3' : 'https://genius.com/Kanye-west-skit-3-lyrics',
                    'Mercy' : 'https://genius.com/Kanye-west-big-sean-pusha-t-and-2-chainz-mercy1-lyrics',
                    'New God Flow' : 'https://genius.com/Kanye-west-pusha-t-and-ghostface-killah-new-god-flow1-lyrics',
                    'Ye vs. the People':'https://genius.com/Kanye-west-ye-vs-the-people-starring-ti-as-the-people-lyrics'}

kanye.iloc[[np.where(kanye['clean_track'] == i)[0][0] for i in list(lookup.keys())],9] =kanye.iloc[[np.where(kanye['clean_track'] == i)[0][0] for i in list(lookup.keys())],7].apply(lambda x: third_search(x))

In [75]:
kanye.loc[kanye['lyrics'] == 'No Lyrics',]
#QA - kanye.to_csv('kanye_qa.csv')

,albums,track_number,track,writers,track_length,Year,old_kanye,clean_track,features,lyrics


In [102]:
kanye.head()

,albums,track_number,track,writers,track_length,Year,old_kanye,clean_track,features,lyrics
0,The College Dropout,1,Intro (Skit),Kanye West,0:19,2004,1,Intro (Skit),None,"\n\n[Intro: DeRay Davis]\nKanye, can I talk to..."
1,The College Dropout,2,We Don't Care,West Miri Ben-Ari Ross Vannelli,3:59,2004,1,We Don't Care,None,\n\n[Intro]\nOh yeah...\nI got the perfect son...
2,The College Dropout,3,Graduation Day,West John Stephens Ben-Ari,1:22,2004,1,Graduation Day,None,"[Spoken: DeRay]\nWhat in the fuck was that, Ka..."
3,The College Dropout,4,All Falls Down (featuring Syleena Johnson),West Lauryn Hill,3:43,2004,1,All Falls Down,Syleena Johnson,"[Chorus: Syleena Johnson & Kanye West]\nOh, wh..."
4,The College Dropout,5,I'll Fly Away,Albert E. Brumley,1:09,2004,1,I'll Fly Away,None,\n\nOne glad morning\nWhen this life is over\n...


In [199]:
kanye.loc[kanye['features'].notnull(),'features'] = kanye.loc[kanye['features'].notnull(),'features'].str.replace('.','').str.replace(
                        ' and', ', ').str.replace('by ', '').str.replace(', ', ',').str.replace(
                        '-', '_').str.replace(', ', ',').str.replace(' ', '_').str.replace(',', ', ')
kanye.loc[kanye['features'].notnull(),'features']

3                                        Syleena_Johnson
5                                       GLC, Consequence
7                                           Jay_Z, J_Ivy
8                                    Talib_Kweli, Common
11                                    Twista, Jamie_Foxx
12                                              Ludacris
17            Mos_Def, Freeway, The_Boys_Choir_of_Harlem
22                                           Adam_Levine
23                                           Lupe_Fiasco
24                                            Jamie_Foxx
26                                        Paul_Wall, GLC
27                                                Common
28                                              The_Game
30                                                Brandy
33                                                 Jay_Z
34                                       Nas, Really_Doe
39                                  Consequence, Cam'ron
46                             

In [203]:
cleaned_feature_artists = kanye.loc[kanye['features'].notnull(),'features'].str.replace('.','').str.replace(
                        ' and', ', ').str.replace('by ', '').str.replace(', ', ',').str.replace(
                        '-', '_').str.replace(', ', ',').str.replace(' ', '_').str.replace(',', ', ')
feature_artists = CountVectorizer()
fit = feature_artists.fit_transform(cleaned_feature_artists)
feature_artists.get_feature_names()[0:5]

['2_chainz', 'adam_levine', 'allan_kingdom', 'big_sean', 'bon_iver']

In [197]:
kanye_top_features_artists = pd.DataFrame(fit.toarray(), columns = feature_artists.get_feature_names()).sum(axis = 0).sort_values(ascending = False)
kanye_top_features_artists.head(n=15)

jay_z             5
pusha_t           4
big_sean          3
paul_mccartney    3
mos_def           2
rick_ross         2
bon_iver          2
common            2
consequence       2
glc               2
jamie_foxx        2
kid_cudi          2
lil_wayne         2
partynextdoor     2
2_chainz          2
dtype: int64

# Jesus is King Addendum

In [113]:
kanye.loc[kanye['albums']=='Jesus is King']

,albums,track_number,track,writers,track_length,Year,old_kanye,clean_track,features,lyrics
126,Jesus is King,1,Every Hour (featuring Sunday Service Choir),Kanye West Benjamin Scholefield Federico Vindver,1:52,2019,0,Every Hour,Sunday Service Choir,NA
127,Jesus is King,2,Selah,West Cydel Young Dexter Raymond Mills Jr. Vind...,2:45,2019,0,Selah,None,NA
128,Jesus is King,3,Follow God,West Gwin Bryant Bell Aaron Butts Curtis Euban...,1:45,2019,0,Follow God,None,NA
129,Jesus is King,4,Closed on Sunday,West Angel Lopez Brian Miller Vindver Timothy ...,2:32,2019,0,Closed on Sunday,None,NA
130,Jesus is King,5,On God,West Gwin Michael Cerda Jordan Timothy Jenks V...,2:16,2019,0,On God,None,NA
131,Jesus is King,6,Everything We Need (featuring Ty Dolla Sign an...,West Tyrone Griffin Jr. Anthony Clemons Michae...,1:56,2019,0,Everything We Need,Ty Dolla Sign and Ant Clemons,NA
132,Jesus is King,7,Water (featuring Ant Clemons),West Clemons Gwin Lopez Vindver Mosley Alexand...,2:48,2019,0,Water,Ant Clemons,NA
133,Jesus is King,8,God Is,West Warryn Campbell Timothy Lee McKenzie Lope...,3:23,2019,0,God Is,None,NA
134,Jesus is King,9,Hands On (featuring Fred Hammond),West Fred Hammond Lopez Vindver Mosley Butts,3:23,2019,0,Hands On,Fred Hammond,NA
135,Jesus is King,10,Use This Gospel (featuring Clipse and Kenny G),West G. Thornton T. Thornton Kenneth Bruce Gor...,3:34,2019,0,Use This Gospel,Clipse and Kenny G,NA


In [116]:
kanye.loc[kanye['albums']=='Jesus is King', 'lyrics'] = kanye.loc[kanye.albums == 'Jesus is King', 'clean_track'].apply(lambda x : lyrics_search(x, 'Kanye West'))
#kanye.loc[kanye['albums']==album, 'lyrics'] = 
#third_search(kanye.iloc[132, 7])
#kanye

Searching for "Every Hour" by Kanye West...
Done.
Searching for "Selah" by Kanye West...
Done.
Searching for "Follow God" by Kanye West...
Done.
Searching for "Closed on Sunday" by Kanye West...
Done.
Searching for "On God" by Kanye West...
Done.
Searching for "Everything We Need" by Kanye West...
Done.
Searching for "Water" by Kanye West...
Done.
Searching for "God Is" by Kanye West...
Done.
Searching for "Hands On" by Kanye West...
Done.
Searching for "Use This Gospel" by Kanye West...
Done.
Searching for "Jesus Is Lord" by Kanye West...
Done.


In [117]:
kanye.loc[kanye['albums']=='Jesus is King']

,albums,track_number,track,writers,track_length,Year,old_kanye,clean_track,features,lyrics
126,Jesus is King,1,Every Hour (featuring Sunday Service Choir),Kanye West Benjamin Scholefield Federico Vindver,1:52,2019,0,Every Hour,Sunday Service Choir,"[Chorus]\nSing every hour (Every hour, 'til th..."
127,Jesus is King,2,Selah,West Cydel Young Dexter Raymond Mills Jr. Vind...,2:45,2019,0,Selah,None,"[Verse 1: Kanye West]\nGod is King, we the sol..."
128,Jesus is King,3,Follow God,West Gwin Bryant Bell Aaron Butts Curtis Euban...,1:45,2019,0,Follow God,None,"[Intro]\nFather, I stretch\nStretch my hands t..."
129,Jesus is King,4,Closed on Sunday,West Angel Lopez Brian Miller Vindver Timothy ...,2:32,2019,0,Closed on Sunday,None,"[Chorus]\nClosed on Sunday, you're my Chick-fi..."
130,Jesus is King,5,On God,West Gwin Michael Cerda Jordan Timothy Jenks V...,2:16,2019,0,On God,None,"[Intro]\nYo, Pi'erre, yo, Pi'erre\n\n[Verse]\n..."
131,Jesus is King,6,Everything We Need (featuring Ty Dolla Sign an...,West Tyrone Griffin Jr. Anthony Clemons Michae...,1:56,2019,0,Everything We Need,Ty Dolla Sign and Ant Clemons,[Intro: Ty Dolla $ign & Ant Clemons]\nWe began...
132,Jesus is King,7,Water (featuring Ant Clemons),West Clemons Gwin Lopez Vindver Mosley Alexand...,2:48,2019,0,Water,Ant Clemons,"[Intro: Kanye West]\nWell, right\nSo, so well\..."
133,Jesus is King,8,God Is,West Warryn Campbell Timothy Lee McKenzie Lope...,3:23,2019,0,God Is,None,"[Intro]\nGod is\nMy light in darkness, oh\nGod..."
134,Jesus is King,9,Hands On (featuring Fred Hammond),West Fred Hammond Lopez Vindver Mosley Butts,3:23,2019,0,Hands On,Fred Hammond,[Refrain: Fred Hammond]\nHand 'em\nHands up hi...
135,Jesus is King,10,Use This Gospel (featuring Clipse and Kenny G),West G. Thornton T. Thornton Kenneth Bruce Gor...,3:34,2019,0,Use This Gospel,Clipse and Kenny G,"[Intro: Kanye West]\nOh, oh, oh, oh, oh, oh\nO..."


In [118]:
kanye.loc[kanye['albums']=='Jesus is King'].to_pickle('kanye_jik.pkl')

In [204]:
kanye.to_pickle('kanye_lyrics.pkl')